# Advanced Tutorial on MetaSynth

In this tutorial, we will be creating a `generative metadata format` (`gmf`) metadata file from a dataset using MetaSynth. We are going to walk through some of the advanced abilities of MetaSynth, such as handling dates, setting distributions and ensuring uniqueness in columns. This example workflow starts from a `.csv` file as input, but it easily adapted to other formats. 

You can run this notebook by checking out the MetaSynth repo and installing metasynth with `pip install .`

In [15]:
%pip install .

Note: you may need to restart the kernel to use updated packages.


ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.
You should consider upgrading via the 'c:\Users\erikj\surfdrive\SoDa\projects\synthetic_data\meta-synth\venv\Scripts\python.exe -m pip install --upgrade pip' command.
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x286e3964460>
c:\Users\erikj\surfdrive\SoDa\projects\synthetic_data\meta-synth\venv\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=4>
  return process_handler(cmd, _system_body)
c:\Users\erikj\surfdrive\SoDa\projects\synthetic_data\meta-synth\venv\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)
c:\Users\erikj\surfdrive\SoDa\projects\synthetic_data\meta-synth\venv\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=6>
  return process_handler(

In [16]:
# import required packages
import datetime as dt
import pandas as pd
from metasynth import MetaDataset, MetaVar
from metasynth.distribution import DiscreteUniformDistribution

## Step 1: Transforming your data into a pandas DataFrame

The first step in creating the metadata is reading and converting your dataset to a pandas DataFrame. 

In [17]:
df = pd.read_csv("demonstration.csv")
df.head()

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,"Braund, Mr. Owen Harris",male,22.0,0,7.2500,NaN,S,1922-04-08,14:31:52.636968,2022-08-14 02:48:29.857190
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,0,71.2833,C85,C,1906-03-26,11:51:04.236820,2022-08-10 10:25:22.268020
2,3,"Heikkinen, Miss. Laina",female,26.0,0,7.9250,NaN,S,1939-09-13,16:56:06.377704,2022-08-13 16:36:00.552160
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,0,53.1000,C123,S,1915-01-03,15:33:58.318343,2022-07-27 19:59:20.559818
4,5,"Allen, Mr. William Henry",male,35.0,0,8.0500,NaN,S,1937-02-09,17:46:53.157645,2022-08-10 02:14:27.126593


MetaSynth will automatically generate the metadata from this DataFrame object so it is important to __ensure the data types for all the variables are correct__. For example, in the dataset above we see that Age is a floating point number whereas it should be an integer (22 instead of 22.0). In addition, there are a few categorical variables (Sex, Parch, Embarked) which are loaded in as string data types.

In general, we support [pandas dtypes](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#basics-dtypes). For our example dataset we can specify the `dtypes` and load the dataset as follows:

In [18]:
dtypes = {
    "Survived": "category",  # Categories should be assigned this type.
    "Name": "string",  # Strings should be assigned like this
    "Age": "Int64",  # Integer columns that have NA's in them should be explicitly nullable integers.
    "Sex": "category",
    "SibSp": "category",
    "Parch": "category",
    "Ticket": "string",
    "Cabin": "string",
    "Embarked": "category",
}
df = pd.read_csv("demonstration.csv", dtype=dtypes)
df.head()

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,"Braund, Mr. Owen Harris",male,22,0,7.2500,<NA>,S,1922-04-08,14:31:52.636968,2022-08-14 02:48:29.857190
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,0,71.2833,C85,C,1906-03-26,11:51:04.236820,2022-08-10 10:25:22.268020
2,3,"Heikkinen, Miss. Laina",female,26,0,7.9250,<NA>,S,1939-09-13,16:56:06.377704,2022-08-13 16:36:00.552160
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,0,53.1000,C123,S,1915-01-03,15:33:58.318343,2022-07-27 19:59:20.559818
4,5,"Allen, Mr. William Henry",male,35,0,8.0500,<NA>,S,1937-02-09,17:46:53.157645,2022-08-10 02:14:27.126593


### Dates, times, and datetimes

One exception to the above is dates, times, and datetimes. Here, we use the types from the built-in `datetime` package. Thus we have to manually transform the strings in the columns with date, time, and datetimes to their proper objects. Since the columns in our example dataset follow the standard ISO-format, we can convert them with the `fromisoformat` method. If they are written in a different format, check out the [datetime library documentation](https://docs.python.org/3/library/datetime.html) on how to convert the strings to datetime/time/date objects.

In [19]:
df["Birthday"] = [dt.date.fromisoformat(x) for x in df["Birthday"]]
df["Board time"] = [dt.time.fromisoformat(x) for x in df["Board time"]]
df["Married since"] = [dt.datetime.fromisoformat(x) for x in df["Married since"]]

Now, let's check the data types of our DataFrame:

In [20]:
df.dtypes

PassengerId               int64
Name                     string
Sex                    category
Age                       Int64
Parch                  category
Fare                    float64
Cabin                    string
Embarked               category
Birthday                 object
Board time               object
Married since    datetime64[ns]
dtype: object

We see that most variables are now nicely specified as strings, categories and ints where necessary. For the dates and times we just created, we see the dtype `object`. This is the "catch-all" dtype for pandas. But don't worry, these columns have the correct type and MetaSynth will deal with it correctly:

In [21]:
df["Birthday"][0]

datetime.date(1922, 4, 8)

## Step 2: Creating a MetaDataset object from a DataFrame

Now a lot of work has already gone into creating a properly formatted dataframe. This work pays off at this stage: let's convert the DataFrame to a meta_dataset structure with the default options. Note: this takes a little bit of time!

In [22]:
meta_dataset = MetaDataset.from_dataframe(df)

c:\users\erikj\surfdrive\soda\projects\synthetic_data\meta-synth\metasynth\disttree.py:119: UserWarning: 
Variable PassengerId seems unique, but not set to be unique.
Set the variable to be either unique or not unique to remove this warning.

  warnings.warn(f"\nVariable {series.name} seems unique, but not set to be unique.\n"


Then, we can show the metadata as a dictionary:

In [23]:
print(meta_dataset)

# Rows: 891
# Columns: 11

{'name': 'PassengerId', 'description': None, 'type': 'MetaVar', 'dtype': 'int64', 'prop_missing': 0.0, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 1, 'high': 892}}"}

{'name': 'Name', 'description': None, 'type': 'MetaVar', 'dtype': 'string', 'prop_missing': 0.0, 'distribution': '.[]{12,82}'}

{'name': 'Sex', 'description': None, 'type': 'MetaVar', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'MultinoulliDistribution', 'parameters': {'labels': array(['female', 'male'], dtype='<U6'), 'probs': array([0.35241302, 0.64758698])}}"}

{'name': 'Age', 'description': None, 'type': 'MetaVar', 'dtype': 'Int64', 'prop_missing': 0.19865319865319866, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 0, 'high': 81}}"}

{'name': 'Parch', 'description': None, 'type': 'MetaVar', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'MultinoulliDistribution', 'parameters': {'lab

### Set unique columns

As you can see, there is one column (PassengerId) that has been detected as unique by MetaSynth. This gives a warning to the user that one might want to add this to the arguments when creating the `meta_dataset`. Without explicitly telling MetaSynth about the uniqueness of columns, MetaSynth will select only non-unique distributions. To surpress the warning one can set the uniqueness of columns to `False`, but in this case we want the `PassengerId` to be unique:

In [25]:
meta_dataset = MetaDataset.from_dataframe(df, spec={"PassengerId": {"unique": True}})
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,n ]1o1pLu7FKB7}8m1xp)9@,male,<NA>,0,10.543728,<NA>,C,1919-08-02,15:34:15.011763,2022-07-18 04:31:48.372256340
1,2,^'7<3#D5it{o7zi7din/eq(6u[qj!=CCKN2}5@O}6Mu2WK...,male,22,0,10.748069,<NA>,S,1918-06-10,17:13:36.487374,2022-08-16 00:03:02.117061060
2,3,"O(f""(Up TW4 J|{[P Ry8h|F]RVR.H4 ^V,Q~Z@p^t""P",female,64,0,35.926007,<NA>,S,1916-08-30,16:04:38.772801,2022-07-24 08:08:43.161545916
3,4,P<)[< tA{+=Jd1s G,female,76,0,48.437912,<NA>,C,1929-11-19,14:00:30.038151,2022-07-24 14:06:23.107410470
4,5,orZUIk.~Wylt,male,70,0,52.659920,<NA>,C,1921-12-09,14:34:32.819233,2022-07-20 20:21:52.165741012


### Fake names (and others)

As one can see, the `Name` of the passengers is not quite so well synthesized. The reason is that MetaSynth by default is able to work well with structured strings (think `R123`). However, it supports the [faker](https://faker.readthedocs.io/en/master/index.html) package, which includes a lot of data types that it can fake. The columns using faker will be completely generative, i.e. they do not use the original data in any way and are thus privacy safe.

We fake names as follows:

In [8]:
meta_dataset = MetaDataset.from_dataframe(
    df,
    unique={"PassengerId": True},
    distribution={"Name": "faker.name"   # Use fake names -> no original data is used.
                 })
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,Diane Perez,male,65,0,8.801308,<NA>,S,1906-05-09,12:25:28.823726,2022-08-15 17:59:57.890636192
1,2,Timothy Lee,male,34,0,18.683875,<NA>,S,1920-01-09,18:12:56.626296,2022-07-28 02:22:43.897241695
2,3,Rachel Barnes,female,<NA>,0,21.413619,<NA>,S,1928-05-28,12:36:28.439398,2022-08-14 17:51:51.140238876
3,4,Angela Jackson,male,36,2,17.301115,<NA>,S,1934-12-23,18:04:09.704319,2022-07-26 13:48:53.062253638
4,5,Derrick Duke,female,29,1,5.695523,448,S,1915-03-01,15:46:49.684362,2022-08-07 02:28:53.358116721


### Set distributions manually

Without user input, the distributions are inferred by choosing the best fitting from available distributions. A user can however also manually set a distribution (either with or without providing the parameters to the distribution).

In [9]:
meta_dataset = MetaDataset.from_dataframe(
    df,
    distribution={"Name": "faker.name",  # Use fake names -> no original data is used.
                   "Fare": "LogNormalDistribution",  # Use a log normal distribution for the Fare, fit parameters.
                   "Age": DiscreteUniformDistribution(20, 40),  # Use uniform distribution, no fitting.
                 },
    unique={"PassengerId": True}  # Force the column 'PassengerId' to be unique.
)
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,Kimberly House,male,36,1,1.103517,<NA>,S,1904-09-28,15:41:26.225594,2022-07-17 03:46:21.230597739
1,2,Brandy Rice,male,28,0,2.172598,B8,S,1929-09-28,12:13:10.154044,2022-08-02 19:51:46.379148512
2,3,Jennifer Mooney,female,<NA>,0,0.289978,<NA>,S,1917-08-02,12:07:34.133282,2022-08-04 21:33:10.087911850
3,4,Todd Washington,female,<NA>,0,7.068193,CBG05,S,1921-02-10,15:20:05.160218,2022-07-15 12:14:27.692548334
4,5,Jennifer Perez,female,24,0,1.760132,<NA>,S,1916-08-14,13:55:08.695122,2022-08-05 18:46:40.118202507


### Set regex distribution manually

For more or less structured strings, we can manually set the structure of the strings. For example, we see that most Cabins are structured like [A-F] and then 2 or 3 digit numbers. We can encapsulate this as follows:

In [12]:
from metasynth.distribution import RegexDistribution

# To create a regex distribution, you need a list of tuples, where each tuple is an element.
# The first part of the tuple is a string representation of the regex, while the second is the proportion of the
# time the regex element is used.
regex = RegexDistribution([(r"[ABCDEF]", 1), (r"\d{2,3}", 0.9)])  # Add the r so that it becomes a literal string.

meta_dataset = MetaDataset.from_dataframe(
    df,
    distribution={"Name": "faker.name",  # Use fake names -> no original data is used.
                   "Fare": "LogNormalDistribution",  # Use a log normal distribution for the Fare, fit parameters.
                   "Age": DiscreteUniformDistribution(20, 40),  # Use uniform distribution, no fitting.
                   "Cabin": regex,  # Manually set regex distribution.
                 },
    unique={"PassengerId": True}  # Force the column 'PassengerId' to be unique.
)
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,Mallory Blair,male,39,0,0.333980,<NA>,S,1935-03-24,13:45:43.605526,2022-07-29 08:58:09.168029519
1,2,Angela Hogan,male,35,0,0.612758,C34,S,1915-12-01,11:49:27.850256,2022-08-14 08:31:33.696420720
2,3,Elizabeth Henderson,male,<NA>,0,0.763967,<NA>,S,1939-11-25,12:32:44.345763,2022-08-11 15:59:02.157057653
3,4,Susan Howard,male,27,0,0.149619,<NA>,S,1922-06-14,18:07:45.747317,2022-07-23 02:27:24.819813276
4,5,Jeremy Williamson,male,33,0,0.459128,<NA>,S,1912-06-10,15:06:42.432252,2022-08-08 03:01:31.229623805


### Privacy package (experimental)

The last feature that is currently experimental is the implementation of privacy features to ensure there is less disclosure. We are working on a [disclosure control](https://github.com/sodascience/metasynth-disclosure-control) extention of MetaSynth that replaces the fitting methods with one that is more safe. 

In [11]:
meta_dataset = MetaDataset.from_dataframe(
    df,
    distribution={"Name": "faker.name",  # Use fake names -> no original data is used.
                 },
    unique={"PassengerId": True},  # Force the column 'PassengerId' to be unique.
    privacy_package="disclosure",  # Use the metasynth-disclosure package (needs to be installed).
)
meta_dataset.synthesize(5)

/home/qubix/Documents/shared_work/synthetic/meta-synth/metasynth/disttree.py:118: UserWarning: 
Variable Age seems unique, but not set to be unique.
Set the variable to be either unique or not unique to remove this warning.

  warnings.warn(f"\nVariable {series.name} seems unique, but not set to be unique.\n"
/home/qubix/.pyenv/versions/3.10.4/lib/python3.10/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/home/qubix/.pyenv/versions/3.10.4/lib/python3.10/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,4,Cassandra Gray,male,2,0,77.868293,<NA>,Q,1912-12-29,14:11:31.089643,2022-08-13 09:24:39.487873818
1,5,Christopher Hughes,female,4,1,250.955985,<NA>,Q,1936-06-17,17:56:42.531979,2022-08-09 01:57:16.488588629
2,6,Edward Santos,female,1,2,20.159560,<NA>,S,1911-06-27,11:15:56.303016,2022-07-27 19:57:17.235885458
3,7,Kenneth Garcia,male,<NA>,2,61.437273,<NA>,S,1923-10-12,12:45:48.840217,2022-07-17 19:10:47.807715953
4,8,Shelby Proctor,female,<NA>,0,2.541149,Port Ricky,S,1933-01-21,10:48:39.910753,2022-08-15 16:09:24.029157483
